In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Lambda

In [6]:
import sys
sys.path.remove('/usr/local/lib/python3.6/dist-packages')

In [11]:
input_len = 18000
reduce_input = keras.Input(shape=(input_len), name="origin_dim")
reduce_output = layers.Dense(50, activation="relu")(reduce_input)
reduce = keras.Model(reduce_input, reduce_output, name="reudce")
reduce.summary()
encoder_input = keras.Input(shape=(input_len * 2), name="original_img")
x1 = Lambda(lambda x: x[:,:input_len])(encoder_input)
x2 = Lambda(lambda x: x[:,input_len:])(encoder_input)
y1 = reduce(x1)
y2 = reduce(x2)
subtracted = keras.layers.Subtract()([y1, y2])
square_layer =  tf.keras.backend.square(subtracted)
sum_layer = tf.keras.backend.sum(square_layer)
encoder = keras.Model(encoder_input, sum_layer, name="encoder")
encoder.summary()
encoder.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={
        tf.keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error")
    },
#    loss_weights=[1.0, 0.2],
)
keras.utils.plot_model(encoder, "multi_input_and_output_model.png", show_shapes=True)
model = encoder


Model: "reudce"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
origin_dim (InputLayer)      [(None, 18000)]           0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                900050    
Total params: 900,050
Trainable params: 900,050
Non-trainable params: 0
_________________________________________________________________
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_img (InputLayer)       [(None, 36000)]      0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 18000)        0           original_img[0][0]               
____

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import json
import collections
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
PATH_CELL_ONTOLOGY = '/raidixshare_log-g/bbrowser/tung/cell_ontology/data/cell'
PATH_CELL_TYPE_DATA = '/raidixshare_log-g/bbrowser/tung/Cell_type_data/'
root = 'BTC000000'


# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

global_data = {}



map_btc2name = {}
for cell_type in os.listdir(PATH_CELL_ONTOLOGY):
    if '.' in cell_type:
        continue
    with open(os.path.join(PATH_CELL_ONTOLOGY, cell_type, 'info.json'), 'r') as fin:
        data = json.load(fin)
    map_btc2name[cell_type] = data["name"]

def get_name_btc(list_btc):
    return [map_btc2name[i] for i in list_btc]
    


class NodeRandomForest:
    _number_train = 20000
    _number_test =  20000
    
    def __init__(self):
        return

    def train(self, node_id, node_data, child_data, list_dir_node):
        if 'BTC000229' != node_id:
            return
        
        all_data = {}
        for child in child_data:
            if len(child_data[child]["train"]) + len(child_data[child]["test"]) <= 3:
                continue
            all_data[child] = child_data[child]
        
        if len(node_data["train"]) != 0:
            all_data[node_id] = node_data
        
        list_dir_node[node_id] = all_data
        
        X_train = []
        Y_train = []
        X_test = []
        Y_test = []
        study_id_train = []
        study_id_test = []
        
        size_matrix = []
        if not len(all_data) >= 2:
            return

        for node in all_data:
            number_study = len(all_data[node]["train"])
            sample_one_study = int(self._number_train / (number_study * len(all_data)))
            for study_dir in all_data[node]["train"]:
                with open(study_dir, 'rb') as fin:
                    #print(study_dir)
                    data = pickle.load(fin)
                    #print([name for name in data])
                    matrix = data['X']
                    sampling = random.choices(range(len(matrix)), k=sample_one_study)
                    X_train.append(matrix[np.array(sampling), :])
                    Y_train.extend([node] * len(sampling))
                    study_id_train.extend([study_dir] * len(sampling))
                    #study_id_train.extend([study_dir.split('/')[-1]] * len(sampling))

            number_study = len(all_data[node]["test"])
            sample_one_study = int(self._number_test / (number_study * len(all_data)))
            for study_dir in all_data[node]["test"]:
                with open(study_dir, 'rb') as fin:
                    data = pickle.load(fin)
                    #print([name for name in data])
                    matrix = data['X']
                    size_matrix.append((len(matrix), node))
                    sampling = random.choices(range(len(matrix)), k=sample_one_study)
                    X_test.append(matrix[np.array(sampling), :])
                    Y_test.extend([node] * len(sampling))
                    study_id_test.extend([study_dir] * len(sampling))
        
        X_train = np.concatenate((X_train), axis=0)
        #X_train = X_train[:, selected_col]
        
        col_sum = sum(X_train != 0) / X_train.shape[0]
        pos = np.where(col_sum > 3.0 / (len(all_data) * 10) )[0]
        print("Selected ", len(pos), 'feaures')
        X_train = X_train[:,pos]
        
        
        X_test = np.concatenate((X_test), axis=0)
        #X_test = X_test[:,selected_col]
        #X_test = X_test[:, selected_col]
        X_test = X_test[:,pos]
        print(Counter(study_id_train))
        

        le = preprocessing.LabelEncoder()                                                                                                                                         
        le.fit(Y_train)
        Y_train = le.transform(Y_train)
        Y_test = le.transform(Y_test)
        
        le_study = preprocessing.LabelEncoder()
        le_study.fit(study_id_train)
        study_id_train = le_study.transform(study_id_train)
        
        
        '''
        clf = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(X_train.shape[1])),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10)
        ])
        clf.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
        
        clf.fit(X_train, Y_train, epochs=10)
        
        probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
        predictions = probability_model.predict(test_images)
        np.argmax(predictions)
        '''
        
        global_data["X_train"] = X_train
        global_data["X_test"] = X_test
        global_data["Y_train"] = Y_train
        global_data["Y_test"] = Y_test
        global_data["study"] = study_id_train
        
        
        
        clf = RandomForestClassifier(n_estimators= 90, max_depth=40,
                                    max_leaf_nodes= max(Y_train + 1) * 20,
                                    random_state=0, n_jobs = 30)
        clf = clf.fit(X_train, Y_train)
        
        train_score = clf.score(X_train, Y_train)
        test_score = clf.score(X_test, Y_test)
        print("###########################################")
        print(node_id)
        print("Score :", train_score)
        print("Test :", test_score)
        
        
        clf = RandomForest(1, max_depth=10)
        clf.fit(X_train, Y_train, study_id_train)
        
        train_score = clf.score(X_train, Y_train)
        test_score = clf.score(X_test, Y_test)
        print("###########################################")
        print(node_id)
        print("Score :", train_score)
        print("Test :", test_score)
        print(size_matrix)
        confusion = confusion_matrix(Y_test, clf.predict(X_test))
        
        
        with open(os.path.join(PATH_CELL_TYPE_DATA, node_id, 'result.pkl'), 'wb') as fout:
            pickle.dump({
              "train_score": train_score,
              "test_score": test_score,
              "confusion_matrix": confusion,
              "label": le.classes_
            }, fout)
        self.plot_confusion(node_id)
        with open(os.path.join(PATH_CELL_TYPE_DATA, node_id, 'model.pkl'), 'wb') as fout:
            pickle.dump({
                "model": clf,
                "le": le
            }, fout)
            
    def remove_result(self, node_id):
        if not os.path.exists(os.path.join(PATH_CELL_TYPE_DATA, node_id, 'result.pkl')):
            return
        os.remove(os.path.join(PATH_CELL_TYPE_DATA, node_id, 'result.pkl'))
        
        
    def plot_confusion(self, node_id):
        
        if not os.path.exists(os.path.join(PATH_CELL_TYPE_DATA, node_id, 'result.pkl')):
            return
        with open(os.path.join(PATH_CELL_TYPE_DATA, node_id, 'result.pkl'), 'rb') as fin:
            data = pickle.load(fin)
            
        print(node_id)
        print("Train score ", data["train_score"])
        print("Test score", data["test_score"])

        plt.rcParams["figure.figsize"] = (10, 10)
        fig, ax = plt.subplots()

        cax = ax.imshow(data["confusion_matrix"], interpolation='nearest')
        #fig.colorbar(cax)
        ax.set_xticks(np.arange(len(data["label"])))
        ax.set_yticks(np.arange(len(data["label"])))

        ax.set_xticklabels(get_name_btc(data["label"]))
        ax.set_yticklabels(get_name_btc(data["label"]))
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
                rotation_mode="anchor")

        for i in range(len(data["label"])):
            for j in range(len(data["label"])):
                text = ax.text(j, i, data["confusion_matrix"][i, j],
                              ha="center", va="center", color="w")
                
        ax.set_title("Confusion matrix")
        fig.tight_layout()

        plt.show()


class GraphRandomForest:
    def __init__(self):
        return

    def check_is_trained(self, node):
        if os.path.exists(os.path.join(PATH_CELL_TYPE_DATA, node, 'result.pkl')):
            return True
        return False

    def train_one_node(self, node, list_child_data, list_dir_node):
        if not os.path.exists(os.path.join(PATH_CELL_TYPE_DATA, node)):
            return
        with open(os.path.join(PATH_CELL_TYPE_DATA, node, 'config.json'), 'r') as fin:
            data = json.load(fin)
        parent_train = [os.path.join(PATH_CELL_TYPE_DATA, node, 'data_index',x.replace('.hdf5', '.pkl')) for x in data['train']]
        parent_test = [os.path.join(PATH_CELL_TYPE_DATA, node, 'data_index',x.replace('.hdf5', '.pkl')) for x in data['test']]
        node_random_forest = NodeRandomForest()
        node_random_forest.train(node, 
                                {"train": parent_train, "test": parent_test},
                                list_child_data,
                                list_dir_node)
            

    def train_DFS(self, parent, list_dir_node):
        data_train, data_test = set(), set()
        info_dir = os.path.join(PATH_CELL_ONTOLOGY, parent, 'info.json')
        with open(info_dir, 'r') as fin:
            info_graph = json.load(fin)    
        if "child" not in info_graph:
            info_graph["child"] = []
        
        list_child_trainable = {}
        for child in info_graph["child"]:
            child_train, child_test = self.train_DFS(child, list_dir_node)
            
            ## REMOVE THIS TO RUN DATA FROM CHILD
            
            #data_train = data_train | child_train
            #data_test = data_test | child_test

            if len(child_train) >= 2 and len(child_test) >= 1:
                list_child_trainable[child] = {"train" : list(child_train), "test": list(child_test)}
        
        if len(list_child_trainable) != 0:
            if not self.check_is_trained(parent):
                self.train_one_node(parent, list_child_trainable, list_dir_node)
        
                
        if os.path.exists(os.path.join(PATH_CELL_TYPE_DATA, parent)):
            with open(os.path.join(PATH_CELL_TYPE_DATA, parent, 'config.json'), 'r') as fin:
                data = json.load(fin)
            parent_train = [os.path.join(PATH_CELL_TYPE_DATA, parent, 'data_index',x.replace('.hdf5', '.pkl')) for x in data['train']]
            parent_test = [os.path.join(PATH_CELL_TYPE_DATA, parent, 'data_index',x.replace('.hdf5', '.pkl')) for x in data['test']]
            data_train  = data_train | set(parent_train)
            data_test = data_test | set(parent_test)

        return data_train, data_test
        

graph = GraphRandomForest()
list_dir_node = {}
ans = graph.train_DFS(root, list_dir_node)

Selected  15743 feaures


NameError: name 'Counter' is not defined